# Voting Records

**Prompt**

Select two Terra Validators of your choice and compare their voting history (governance votes). Off-chain data can be used to support your answer, but you must also utilize Flipside data in your answer to be eligible for full payment.

BONUS: Can you find any proposals where your chosen validators voted differently? Why do you think this might be the case?

## Pull Data

Using the `terra.validator_voting_power` table, I found two validators with a relatively high amount of voting power who had cool names to dig into: **'Luna Station 88’** and **‘moonshot’**

In [1]:
import emoji
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from urllib.request import urlopen

```SQL
WITH ls AS (
  SELECT
  VOTER,
  'Luna Station 88' AS label,
  PROPOSAL_ID,
  OPTION
  FROM terra.gov_vote
  WHERE voter = 'terra1j27nm2gjm0m4lsye8lspa46rax0rw4fge9awrs'
), ms AS (
  SELECT
  VOTER,
  'moonshot' AS label,
  PROPOSAL_ID,
  OPTION
  FROM terra.gov_vote
  WHERE voter = 'terra1xuxwc867udr9k84zzqhaflf9sc00svvmjm4geh'
)
SELECT
CASE WHEN ls.proposal_id IS NULL THEN ms.proposal_id ELSE ls.proposal_id END as proposalId,
ls.label AS ls_label,
ls.option AS ls_option,
ms.label AS ms_label,
ms.option AS ms_option
FROM ls
FULL OUTER JOIN ms ON ms.proposal_id = ls.proposal_id
ORDER BY proposalId DESC
```

In [2]:
# Pull query data from API 

url = 'https://api.flipsidecrypto.com/api/v2/queries/11bc80f7-57c0-4fdd-b9dc-c173bd39742e/data/latest'

response = urlopen(url)
data_json = json.loads(response.read())
df = pd.DataFrame(data_json)
df['PROPOSALID'] = pd.to_numeric(df['PROPOSALID'])
df.sort_values('PROPOSALID', inplace = True)

In [3]:
df.head()

,LS_LABEL,LS_OPTION,MS_LABEL,MS_OPTION,PROPOSALID
15,Luna Station 88,Yes,None,None,53
14,Luna Station 88,Yes,None,None,54
13,Luna Station 88,Yes,None,None,56
12,Luna Station 88,Yes,None,None,58
11,Luna Station 88,Abstain,moonshot,Yes,67


In [4]:
option_dict ={'Abstain': ':white_large_square:',
              'No': ':cross_mark:',
              None: ':white_large_square:',
              'NoWithVeto': ':cross_mark:',
              'VOTE_OPTION_ABSTAIN': ':white_large_square:',
              'VOTE_OPTION_NO': ':cross_mark:',
              'VOTE_OPTION_YES': ':check_mark_button:',
              'Yes': ':check_mark_button:'}

In [5]:
def format_proposal_id(id):
    length = len(str(id))
    spaces = 12 - length
    return str(id) + ' '*spaces +'|'

## Visualize Votes

In [6]:
print('PROPOSAL_ID | Luna Station 88 | moonshot')
for index, row in df.iterrows():
    print(emoji.emojize(format_proposal_id(row['PROPOSALID']) + \
          '        ' + option_dict[row['LS_OPTION']] + '      |'\
          '    ' + option_dict[row['MS_OPTION']]))

PROPOSAL_ID | Luna Station 88 | moonshot
53          |        ✅      |    ⬜
54          |        ✅      |    ⬜
56          |        ✅      |    ⬜
58          |        ✅      |    ⬜
67          |        ⬜      |    ✅
68          |        ✅      |    ⬜
76          |        ✅      |    ✅
78          |        ✅      |    ✅
82          |        ❌      |    ❌
83          |        ✅      |    ✅
86          |        ❌      |    ✅
90          |        ✅      |    ✅
92          |        ✅      |    ✅
95          |        ✅      |    ⬜
96          |        ⬜      |    ❌
98          |        ✅      |    ✅
100         |        ✅      |    ✅
106         |        ✅      |    ✅
107         |        ⬜      |    ✅
107         |        ⬜      |    ✅
108         |        ✅      |    ✅
110         |        ✅      |    ✅
115         |        ✅      |    ✅
116         |        ✅      |    ✅
117         |        ✅      |    ✅
118         |        ✅      |    ⬜
119         |        ✅      |    ✅
121         | 

## Voting Disagreements

In [7]:
# Just the disagreements
print('PROPOSAL_ID | Luna Station 88 | moonshot')
for index, row in df[df['PROPOSALID'].isin([86, 135, 160])].iterrows():
    print(emoji.emojize(format_proposal_id(row['PROPOSALID']) + \
          '        ' + option_dict[row['LS_OPTION']] + '      |'\
          '    ' + option_dict[row['MS_OPTION']]))

PROPOSAL_ID | Luna Station 88 | moonshot
86          |        ❌      |    ✅
135         |        ✅      |    ❌
160         |        ❌      |    ✅


[Terra proposal 86](https://station.terra.money/proposal/86) was to "Increase the maximum num of validators". Luna Station voted against the proposal while moonshot voted for. The proposal passed.

[Terra proposal 135](https://station.terra.money/proposal/135) was "Community Grant Proposal to Expand ‘TeFi Alpha’ Worldwide". Luna Station voted for the proposal while moonshot voted against. The proposal passed.

[Terra proposal 160](https://station.terra.money/proposal/160) was “Community Grant Proposal: LunaFest Decentralised Crypto Festival". Luna Station voted against the proposal while moonshot voted for. The proposal was rejected.